In [1]:
from sentence_transformers import SentenceTransformer 
from transformers import AutoTokenizer, AutoModel
import numpy as np
#tokenizer = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [2]:
text = ["Introducción a la Programación","Literatura Española", "Álgebra Lineal","Gramatica Inglesa", "Estadística Avanzada","Literatura Inglesa","Matematicas avanzaadas","Historia de Bolivia","Probabilidad matematica"]
context = ["Matematicas, Probabilidad, Logica","Literatura, Historia, Gramatica"]

In [3]:
from torch.utils.data import Dataset, DataLoader
class DatasetCurso(Dataset):
    def __init__(self,data,tokenizer, context):
        self.data = data
        self.tokenizer = tokenizer
        self.context= context
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        ctx = {}
        if idx % 2==0:
            ctx['positivo'] = self.context[0]
            ctx['negativo'] = self.context[1]
            ctx['tipo'] = 'm'
            
        else:
            ctx['positivo'] = self.context[1]
            ctx['negativo'] = self.context[0]
            ctx['tipo'] = 'h'
        
        it = self.data[idx]
        ctx['positivo'] = tokenizer(ctx['positivo'], return_tensors='pt')
        ctx['negativo'] = tokenizer(ctx['negativo'],return_tensors='pt')
        it = tokenizer(it, return_tensors='pt')
        return it,ctx

In [4]:
dt = DatasetCurso(text,tokenizer,context)
train_dataloader = DataLoader(dt, batch_size=1)

In [5]:
import torch.nn as nn
import torch
epoch = 10
cosine_loss = nn.CosineEmbeddingLoss(margin=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-7)

In [6]:
for i in range(epoch):
    for target,ctx in train_dataloader:
        positiveSentence = ctx['positivo']
        negativeSentence = ctx['negativo']
        embeddingST = model(input_ids=positiveSentence['input_ids'][0],attention_mask=positiveSentence['attention_mask'])
        embeddingNT = model(input_ids=negativeSentence['input_ids'][0],attention_mask=negativeSentence['attention_mask'])
        embeddingT = model(input_ids=target['input_ids'][0],attention_mask=target['attention_mask'])
        cossineL = cosine_loss(embeddingST.pooler_output, embeddingT.pooler_output,torch.tensor([1.0])) + cosine_loss(embeddingNT.pooler_output, embeddingT.pooler_output, torch.tensor([-1.0]))
        print(cossineL)
        optimizer.zero_grad()
        cossineL.backward()
        optimizer.step()

tensor(0.7179, grad_fn=<AddBackward0>)
tensor(0.3444, grad_fn=<AddBackward0>)
tensor(0.4970, grad_fn=<AddBackward0>)
tensor(0.4665, grad_fn=<AddBackward0>)
tensor(0.5405, grad_fn=<AddBackward0>)
tensor(0.2329, grad_fn=<AddBackward0>)
tensor(0.4023, grad_fn=<AddBackward0>)
tensor(0.6165, grad_fn=<AddBackward0>)
tensor(0.1025, grad_fn=<AddBackward0>)
tensor(0.6605, grad_fn=<AddBackward0>)
tensor(0.2917, grad_fn=<AddBackward0>)
tensor(0.4370, grad_fn=<AddBackward0>)
tensor(0.4256, grad_fn=<AddBackward0>)
tensor(0.4978, grad_fn=<AddBackward0>)
tensor(0.1908, grad_fn=<AddBackward0>)
tensor(0.3431, grad_fn=<AddBackward0>)
tensor(0.5923, grad_fn=<AddBackward0>)
tensor(0.0970, grad_fn=<AddBackward0>)
tensor(0.6220, grad_fn=<AddBackward0>)
tensor(0.2409, grad_fn=<AddBackward0>)
tensor(0.4043, grad_fn=<AddBackward0>)
tensor(0.3855, grad_fn=<AddBackward0>)
tensor(0.4565, grad_fn=<AddBackward0>)
tensor(0.1494, grad_fn=<AddBackward0>)
tensor(0.2861, grad_fn=<AddBackward0>)
tensor(0.5718, grad_fn=<A

In [11]:
model.save_pretrained('modelSemanticHugging')
tokenizer.save_pretrained('modelSemanticHugging')

('modelSemanticHugging\\tokenizer_config.json',
 'modelSemanticHugging\\special_tokens_map.json',
 'modelSemanticHugging\\sentencepiece.bpe.model',
 'modelSemanticHugging\\added_tokens.json',
 'modelSemanticHugging\\tokenizer.json')

In [22]:
from sentence_transformers import SentenceTransformer, models

#ruta_modelo_huggingface = "modelSemantic"
#modeloCargado = torch.load(ruta_modelo_huggingface)


transformador_huggingface = models.Transformer('modelSemanticHugging')
pooling_model = models.Pooling(768,
                               pooling_mode_mean_tokens=True,
                               pooling_mode_max_tokens=False)
modelo_sentence_transformer = SentenceTransformer(modules=[])

modelo_sentence_transformer.add_module("transformer", transformador_huggingface)
modelo_sentence_transformer.add_module("pooling",pooling_model)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
modelo_pretrain = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

## Modelo Pretrain vs Fine Tuning

In [35]:
text = ["Álgebra Lineal", "Estadística Avanzada","Literatura Española",]
encoded_modelpretrained = modelo_pretrain.encode(text)
encoded_modelFineTune = modelo_sentence_transformer.encode(text)
print("Model Pretrain")

print("Algebra, Estadistica similitud:", cosine_similarity([encoded_modelpretrained[0]],[encoded_modelpretrained[1]]))
print("Algebra, Literatura Española similitud:", cosine_similarity([encoded_modelpretrained[0]],[encoded_modelpretrained[2]]))
print('-----------------------')
print("Model Fine Tune")
print("Algebra, Estadistica similitud:", cosine_similarity([encoded_modelFineTune[0]],[encoded_modelFineTune[1]]))
print("Algebra, Literatura española similitud:", cosine_similarity([encoded_modelFineTune[0]],[encoded_modelFineTune[2]]))

Model Pretrain
Algebra, Estadistica similitud: [[0.22193216]]
Algebra, Literatura Española similitud: [[0.17139134]]
-----------------------
Model Fine Tune
Algebra, Estadistica similitud: [[0.32248306]]
Algebra, Literatura española similitud: [[0.10733536]]


## More examples

In [36]:
text = ["Introducción a la Programación","Literatura Española", "Álgebra Lineal","Gramatica Inglesa", "Estadística Avanzada","Literatura Inglesa","Matematicas avanzaadas","Historia de Bolivia","Probabilidad matematica"]


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
for a, b in zip(text, modelo_pretrain.encode(text)):
    for j, k in zip(text, modelo_pretrain.encode(text)):
        print(f"Nombre del curso en español: {a}", "---",j)
        print("similitud:", cosine_similarity([b],[k]))
        
        print("\n")

Nombre del curso en español: Introducción a la Programación --- Introducción a la Programación
similitud: [[1.]]


Nombre del curso en español: Introducción a la Programación --- Literatura Española
similitud: [[0.17036648]]


Nombre del curso en español: Introducción a la Programación --- Álgebra Lineal
similitud: [[0.31900138]]


Nombre del curso en español: Introducción a la Programación --- Gramatica Inglesa
similitud: [[0.27318677]]


Nombre del curso en español: Introducción a la Programación --- Estadística Avanzada
similitud: [[0.26336113]]


Nombre del curso en español: Introducción a la Programación --- Literatura Inglesa
similitud: [[0.14597407]]


Nombre del curso en español: Introducción a la Programación --- Matematicas avanzaadas
similitud: [[0.48074913]]


Nombre del curso en español: Introducción a la Programación --- Historia de Bolivia
similitud: [[0.10130028]]


Nombre del curso en español: Introducción a la Programación --- Probabilidad matematica
similitud: [[0.19

In [38]:
for a, b in zip(text, modelo_sentence_transformer.encode(text)):
    for j, k in zip(text, modelo_sentence_transformer.encode(text)):
        print(f"Nombre del curso en español: {a}", "---",j)
        print("similitud:", cosine_similarity([b],[k]))
        
        print("\n")

Nombre del curso en español: Introducción a la Programación --- Introducción a la Programación
similitud: [[1.0000001]]


Nombre del curso en español: Introducción a la Programación --- Literatura Española
similitud: [[0.11403426]]


Nombre del curso en español: Introducción a la Programación --- Álgebra Lineal
similitud: [[0.419146]]


Nombre del curso en español: Introducción a la Programación --- Gramatica Inglesa
similitud: [[0.16262013]]


Nombre del curso en español: Introducción a la Programación --- Estadística Avanzada
similitud: [[0.32770827]]


Nombre del curso en español: Introducción a la Programación --- Literatura Inglesa
similitud: [[0.09903692]]


Nombre del curso en español: Introducción a la Programación --- Matematicas avanzaadas
similitud: [[0.55693126]]


Nombre del curso en español: Introducción a la Programación --- Historia de Bolivia
similitud: [[0.08639112]]


Nombre del curso en español: Introducción a la Programación --- Probabilidad matematica
similitud: [